In [1]:
# Mount Google drive to upload datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# The path to the data on my drive
D = '/content/drive/My Drive/W266_Project_Data/pmi_data'

In [4]:
import pandas as pd
import numpy as np

In [22]:
# Load the eval files for each model's translations
bart_eval = pd.read_csv(D+"/predicted_text/siamese_evaluations_of_translations/bart_siamese_paraphrase_eval.csv")
bart_eval.drop(columns={'Unnamed: 0'}, inplace=True)
indictrans_eval = pd.read_csv(D+"/predicted_text/siamese_evaluations_of_translations/indictrans_siamese_paraphrase_eval.csv")
indictrans_eval.drop(columns={'Unnamed: 0'}, inplace=True)

In [23]:
# View the first few records in each file
bart_eval.head()

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,0.088514,0.770237
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,0.979257,0.946777
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,0.986500,0.956433
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.980670,0.965325
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...",0.984052,0.979276


In [24]:
indictrans_eval.head()

,language_task,target_sentence,indictrans_translation,probability_of_paraphrase,cosine_similarity_scores
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,0.974685,0.948379
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...,0.978843,0.973164
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.987696,0.985965
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.986279,0.986373
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो...",0.985711,0.992806


In [25]:
result = pd.merge(bart_eval, indictrans_eval, how="outer", on=["target_sentence"])

In [26]:
# View several rows
result

,language_task_x,target_sentence,bart_translation,probability_of_paraphrase_x,cosine_similarity_scores_x,language_task_y,indictrans_translation,probability_of_paraphrase_y,cosine_similarity_scores_y
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,0.088514,0.770237,translate English to Hindi,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,0.974685,0.948379
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,0.979257,0.946777,translate English to Hindi,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...,0.978843,0.973164
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,0.986500,0.956433,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.987696,0.985965
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.980670,0.965325,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.986279,0.986373
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...",0.984052,0.979276,translate English to Hindi,"इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो...",0.985711,0.992806
...,...,...,...,...,...,...,...,...,...
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,vi. மியான்மர் மத்திய தேர்தல் ஆணையம்;,0.967841,0.911552,translate English to Tamil,மியான்மர் ஒன்றிய தேர்தல் ஆணையம் மற்றும்,0.601117,0.760211
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,சுதந்திரத்திற்குப் பிறகு பல சாதனைகள் ஏற்பட்டுள...,0.114865,0.878720,translate English to Tamil,நாடு சுதந்திரம் அடைந்ததிலிருந்து பல சாதனைகள் ப...,0.089361,0.886990
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...",ஜன் பாகிதாரி உள்ளிட்ட நல்ல நிர்வாகத்தின் சில அ...,0.392906,0.932954,translate English to Tamil,"மக்கள் பங்கேற்பு, தகவல் பரிமாற்றம், ஆதாரங்களை ...",0.741024,0.961721
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","கிராம சுயாட்சி இயக்கம், ஆயுஷ்மான் பாரத் போன்ற ...",0.108764,0.942588,translate English to Tamil,அண்மையில் மேற்கொள்ளப்பட்ட கிராம சுயாட்சி இயக்க...,0.088348,0.935723


In [27]:
# The language task field is also the same everywhere - we'll drop one instance of this column and rename the other
result['language_task_x'].equals(result['language_task_y'])

True

In [28]:
result.columns

Index(['language_task_x', 'target_sentence', 'bart_translation',
       'probability_of_paraphrase_x', 'cosine_similarity_scores_x',
       'language_task_y', 'indictrans_translation',
       'probability_of_paraphrase_y', 'cosine_similarity_scores_y'],
      dtype='object')

In [29]:
# Drop the redundant column
result.drop(columns={'language_task_y'}, inplace=True)

# Rename the columns of interest
result.rename(columns={'language_task_x': 'language_task', 'probability_of_paraphrase_x': 'bart_paraphrase_score',
                       'cosine_similarity_scores_x': 'bart_cosine_score', 'probability_of_paraphrase_y': 'indictrans_paraphrase_score',
                       'cosine_similarity_scores_y': 'indictrans_cosine_score'}, inplace=True)

In [30]:
result.head()

,language_task,target_sentence,bart_translation,bart_paraphrase_score,bart_cosine_score,indictrans_translation,indictrans_paraphrase_score,indictrans_cosine_score
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,0.088514,0.770237,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,0.974685,0.948379
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,0.979257,0.946777,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...,0.978843,0.973164
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,0.986500,0.956433,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.987696,0.985965
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.980670,0.965325,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.986279,0.986373
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...",0.984052,0.979276,"इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो...",0.985711,0.992806


In [31]:
result.tail()

,language_task,target_sentence,bart_translation,bart_paraphrase_score,bart_cosine_score,indictrans_translation,indictrans_paraphrase_score,indictrans_cosine_score
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,vi. மியான்மர் மத்திய தேர்தல் ஆணையம்;,0.967841,0.911552,மியான்மர் ஒன்றிய தேர்தல் ஆணையம் மற்றும்,0.601117,0.760211
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,சுதந்திரத்திற்குப் பிறகு பல சாதனைகள் ஏற்பட்டுள...,0.114865,0.878720,நாடு சுதந்திரம் அடைந்ததிலிருந்து பல சாதனைகள் ப...,0.089361,0.886990
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...",ஜன் பாகிதாரி உள்ளிட்ட நல்ல நிர்வாகத்தின் சில அ...,0.392906,0.932954,"மக்கள் பங்கேற்பு, தகவல் பரிமாற்றம், ஆதாரங்களை ...",0.741024,0.961721
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","கிராம சுயாட்சி இயக்கம், ஆயுஷ்மான் பாரத் போன்ற ...",0.108764,0.942588,அண்மையில் மேற்கொள்ளப்பட்ட கிராம சுயாட்சி இயக்க...,0.088348,0.935723
2999,translate English to Tamil,இந்தியாவின் 130 லட்சம் மக்கள் சார்பில் உங்கள் ...,இந்தியாவின் மூன்று பில்லியன் மக்களின் சார்பில்...,0.082759,0.942548,இந்தியாவின் 130 கோடி மக்களின் சார்பாக உங்கள் அ...,0.944056,0.979156


In [32]:
# Check if any of the paraphrase scores are exactly equal
len(result[result.bart_paraphrase_score == result.indictrans_paraphrase_score])

33

In [33]:
# Create a new column using the "best" translation as measured by paraphrase score
# For equal sentences (IndicTrans and MBart produced the same output, we'll keep the Bart record)
result['best_trans'] = np.where(result.indictrans_paraphrase_score >= result.bart_paraphrase_score, result.indictrans_translation, result.bart_translation)

In [38]:
# Number of times that Bart produced a better translation than Indictrans
len(result[result.best_trans == result.bart_translation]) - 33

1079

In [39]:
# Number of times that Indictrans produced a better translation than Bart
len(result[result.best_trans == result.indictrans_translation]) - 33

1888

In [40]:
# Filter the dataset into its respective languages and save the target and results to compute language level sacre bleu
hi_result = result[result.language_task == 'translate English to Hindi']
ta_result = result[result.language_task == 'translate English to Tamil']
ml_result = result[result.language_task == 'translate English to Malayalam']

In [43]:
# Pringt the number of times for each language when the models produced translations with the same paraphrase score
print(len(hi_result[hi_result.bart_paraphrase_score == hi_result.indictrans_paraphrase_score]))
print(len(ta_result[ta_result.bart_paraphrase_score == ta_result.indictrans_paraphrase_score]))
print(len(ml_result[ml_result.bart_paraphrase_score == ml_result.indictrans_paraphrase_score]))

24
9
0


In [45]:
# Find the number of times that MBart produced a better translation for Hindi than IndicTrans and vice versa
print(len(hi_result[hi_result.best_trans == hi_result.bart_translation]) - len(hi_result[hi_result.bart_paraphrase_score == hi_result.indictrans_paraphrase_score]))
print(len(hi_result[hi_result.best_trans == hi_result.indictrans_translation]) - len(hi_result[hi_result.bart_paraphrase_score == hi_result.indictrans_paraphrase_score]))

340
636


In [46]:
# Find the number of times that MBart produced a better translation for Tamil than IndicTrans and vice versa
print(len(ta_result[ta_result.best_trans == ta_result.bart_translation]) - len(ta_result[ta_result.bart_paraphrase_score == ta_result.indictrans_paraphrase_score]))
print(len(ta_result[ta_result.best_trans == ta_result.indictrans_translation]) - len(ta_result[ta_result.bart_paraphrase_score == ta_result.indictrans_paraphrase_score]))

459
532


In [47]:
# Find the number of times that MBart produced a better translation for Malayalam than IndicTrans and vice versa
print(len(ml_result[ml_result.best_trans == ml_result.bart_translation]) - len(ml_result[ml_result.bart_paraphrase_score == ml_result.indictrans_paraphrase_score]))
print(len(ml_result[ml_result.best_trans == ml_result.indictrans_translation]) - len(ml_result[ml_result.bart_paraphrase_score == ml_result.indictrans_paraphrase_score]))

280
720


In [48]:
# Save the target translations and the best translations for each language to a separate txt file
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/hi_target.txt", hi_result.target_sentence.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/hi_best.txt", hi_result.best_trans.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ta_target.txt", ta_result.target_sentence.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ta_best.txt", ta_result.best_trans.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ml_target.txt", ml_result.target_sentence.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ml_best.txt", ml_result.best_trans.values, fmt='%s', newline='\n')

In [49]:
# cd into the directory where the files are saved then compute Sacre Bleu scores
%cd /content/drive/MyDrive/W266_Project_Data/pmi_data/predicted_text/siamese_evaluations_of_translations/

/content/drive/MyDrive/W266_Project_Data/pmi_data/predicted_text/siamese_evaluations_of_translations


In [50]:
# Check for the output txt files
%ls

aryan_test.csv
bart_siamese_paraphrase_eval.csv
bart_siamese_paraphrase_eval_v2.csv
drav_test.csv
drav_test.gsheet
hi_best.txt
hi_target.txt
indictrans_siamese_paraphrase_eval.csv
ml_best.txt
ml_target.txt
paraphrase_test_eval_and_errors.csv
paraphrase_test_eval_and_errors.gsheet
record_comparison.csv
ta_best.txt
ta_target.txt
three_dense_paraphrase_test_eval_and_errors.csv
three_dense_w_punjabi_paraphrase_test_eval_and_errors.csv
three_dense_w_punjabi_paraphrase_test_eval_and_errors.gsheet
three_dense_w_punjabi_paraphrase_test_eval_and_errors_schedule.csv
two_dense_w_punjabi_paraphrase_test_eval_and_errors.csv


In [51]:
!pip install sacrebleu -q

     |████████████████████████████████| 90 kB 6.1 MB/s 


In [52]:
# Get the Hindi Sacre Bleu scores
!sacrebleu hi_best.txt -i hi_target.txt --tokenize intl

{
 "name": "BLEU",
 "score": 38.4,
 "signature": "nrefs:1|case:mixed|eff:no|tok:intl|smooth:exp|version:2.0.0",
 "verbose_score": "67.5/45.4/31.5/22.4 (BP = 1.000 ratio = 1.023 hyp_len = 19557 ref_len = 19108)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "intl",
 "smooth": "exp",
 "version": "2.0.0"
}


In [53]:
# Get the Tamil translation Sacre Bleu scores
!sacrebleu ta_best.txt -i ta_target.txt --tokenize intl

{
 "name": "BLEU",
 "score": 15.6,
 "signature": "nrefs:1|case:mixed|eff:no|tok:intl|smooth:exp|version:2.0.0",
 "verbose_score": "45.6/20.7/10.7/5.8 (BP = 1.000 ratio = 1.048 hyp_len = 14425 ref_len = 13763)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "intl",
 "smooth": "exp",
 "version": "2.0.0"
}


In [54]:
# Get the Malayalam translation Sacre Bleu scores
!sacrebleu ml_best.txt -i ml_target.txt --tokenize intl

{
 "name": "BLEU",
 "score": 7.2,
 "signature": "nrefs:1|case:mixed|eff:no|tok:intl|smooth:exp|version:2.0.0",
 "verbose_score": "35.2/10.7/4.4/1.9 (BP = 0.961 ratio = 0.962 hyp_len = 11929 ref_len = 12402)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "intl",
 "smooth": "exp",
 "version": "2.0.0"
}


In [55]:
# Save the result file
result.to_csv(D+"/predicted_text/siamese_evaluations_of_translations/record_comparison.csv")